## Load and get familiar with the data

Run the following to get the data

```
git clone https://github.com/bsmith89/statsclub_pymc3
cd statsclub_pymc3
make data/clean_data.tsv
```

We'll load this cleaned dataset.

In [ ]:
import pandas as pd

data = pd.read_table('data/clean_data.tsv')
data.shape

In [ ]:
data.head()

Let's take a look at the distribution of radon measurements.

In [ ]:
import matplotlib.pyplot as plt

_ = plt.hist(data.radon, bins=100)
plt.yscale('symlog')

Clearly a skewed distribution, and bounded at 0.

We would normally think to log-transform these data, but we have a bunch of
observations that were below the detection limit (0.1 of whatever units these measurements are in).

In [ ]:
data[data.radon != 0].radon.min()

## Data transformations

We'll replace those zeros with half of the detection limit, purely as a heuristic.

In [ ]:
data['radon_nonzero'] = data.radon
data.radon_nonzero[data.radon == 0] = 0.05

Now we can see that these data are approximately normally distributed.

Much better for our purposes.

In [ ]:
import numpy as np

_ = plt.hist(data.radon_nonzero.apply(np.log), bins=100)

In [ ]:
data['log_radon'] = data['radon_nonzero'].apply(np.log)

To make our analysis easier to manage, we'll start by just looking at measurements from Minnesota.

In [ ]:
d = data[data.state == 'MN']

The first factor that we're going to consider for predicting radon levels is
where the measurement was taken.

We know that radon comes out of the ground, so it makes sense that levels are
higher in basements compared to the rest of a house.

That's approximately what we see.

In [ ]:
plt.scatter('is_basement', 'log_radon', data=d, alpha=0.5)

In [ ]:
import seaborn as sns

sns.violinplot('is_basement', 'log_radon', data=d)

## Classic linear model (Complete pooling)

In [ ]:
import patsy

y, x = patsy.dmatrices('log_radon ~ is_basement', data=d, return_type='dataframe')
n, r = x.shape

In [ ]:
import statsmodels.api as sm

fit0 = sm.OLS(y, x).fit()

fit0.summary()

In [ ]:
def jitter(x, perc=0.05):
    span = max(x) - min(x)
    return x + (np.random.rand(len(x)) - 0.5) * perc * span

In [ ]:
plt.scatter(jitter(fit0.predict(), 0.1),
            fit0.resid_pearson, s=0.5)

In [ ]:
d['predict0'] = fit0.predict()
d['resid0'] = d.predict0 - d.log_radon

sns.violinplot('is_basement', 'resid0', data=d)

In [ ]:
plt.scatter(jitter(d['is_basement']), d['log_radon'], s=1)
plt.plot([0, 1], [fit0.params[0], fit0.params[0] + fit0.params[1]], color='k')

## Bayesian version complete pooling model

In [ ]:
import pymc3 as pm
import theano.tensor as tt

with pm.Model() as model0:
    beta = pm.Normal('beta', sd=10, shape=(r, 1))
    sigma = pm.HalfCauchy('sigma', beta=2)
    
    mu = tt.dot(x.values, beta)
    
    obs = pm.Normal('obs', mu=mu, sd=sigma, observed=y)

In [ ]:
model0.logp(model0.test_point)

In [ ]:
with model0:
    trace0 = pm.sample(tune=1000)

In [ ]:
pm.traceplot(trace0)

## No pooling model

In [ ]:
counties = d.state_county.unique()
county_lookup = dict(zip(counties, range(len(counties))))
d['county_idx'] = d.state_county.replace(county_lookup)

In [ ]:
y, x1 = patsy.dmatrices('log_radon ~ is_basement', data=d, return_type='dataframe')
x1.drop(columns=['Intercept'], inplace=True)
n, r1 = x1.shape

x2 = patsy.dmatrix('state_county - 1', data=d, return_type='dataframe')
r2 = x2.shape[1]


with pm.Model() as model1:
    beta = pm.Normal('beta', sd=10, shape=(r1, 1))
    gamma = pm.Normal('gamma', sd=10, shape=(r2, 1))
    sigma = pm.HalfCauchy('sigma', beta=2)
    
    mu = tt.dot(x1.values, beta) + tt.dot(x2.values, gamma)
    
    obs = pm.Normal('obs', mu=mu, sd=sigma, observed=y)

In [ ]:
with model1:
    trace1 = pm.sample(tune=1000)

In [ ]:
pm.traceplot(trace1, var_names=['beta', 'sigma'])

In [ ]:
pm.forestplot(trace1, var_names=['gamma'])

## Partial pooling model

In [ ]:
y, x1 = patsy.dmatrices('log_radon ~ is_basement', data=d, return_type='dataframe')
x1.drop(columns=['Intercept'], inplace=True)
n, r1 = x1.shape

x2 = patsy.dmatrix('state_county - 1', data=d, return_type='dataframe')
r2 = x2.shape[1]


with pm.Model() as model2:
    beta = pm.Normal('beta', sd=10, shape=(r1, 1))
    
    gamma_hyper_mean = pm.Normal('gamma_hyper_mean', sd=10)
    gamma_hyper_sd = pm.HalfCauchy('gamma_hyper_sd', beta=2)
    gamma = pm.Normal('gamma', mu=gamma_hyper_mean, sd=gamma_hyper_sd, shape=(r2, 1))
    
    sigma = pm.HalfCauchy('sigma', beta=2)
    
    mu = tt.dot(x1.values, beta) + tt.dot(x2.values, gamma)
    
    obs = pm.Normal('obs', mu=mu, sd=sigma, observed=y)

In [ ]:
with model2:
    trace2 = pm.sample(tune=1000)

In [ ]:
pm.traceplot(trace2, var_names=['beta', 'sigma', 'gamma_hyper_mean', 'gamma_hyper_sd'])

## Reparameterize partial pooling model

In [ ]:
y, x1 = patsy.dmatrices('log_radon ~ is_basement', data=d, return_type='dataframe')
x1.drop(columns=['Intercept'], inplace=True)
n, r1 = x1.shape

x2 = patsy.dmatrix('state_county - 1', data=d, return_type='dataframe')
r2 = x2.shape[1]


with pm.Model() as model3:
    beta = pm.Normal('beta', sd=10, shape=(r1, 1))
    
    gamma_hyper_mean = pm.Normal('gamma_hyper_mean', sd=10)
    gamma_hyper_sd = pm.HalfCauchy('gamma_hyper_sd', beta=2)
    gamma_ = pm.Normal('gamma_', shape=(r2, 1))
    gamma = pm.Deterministic('gamma', gamma_hyper_mean + gamma_hyper_sd * gamma_)
    
    sigma = pm.HalfCauchy('sigma', beta=2)

    mu = tt.dot(x1.values, beta) + tt.dot(x2.values, gamma)
    
    obs = pm.Normal('obs', mu=mu, sd=sigma, observed=y)

In [ ]:
with model3:
    trace3 = pm.sample(tune=1000)

In [ ]:
pm.traceplot(trace3, var_names=['beta', 'sigma', 'gamma_hyper_mean', 'gamma_hyper_sd'])

## Add county-level uranium as predictor

In [ ]:
y, x1 = patsy.dmatrices('log_radon ~ is_basement', data=d, return_type='dataframe')
x1.drop(columns=['Intercept'], inplace=True)
n, r1 = x1.shape

x2 = patsy.dmatrix('state_county - 1', data=d, return_type='dataframe')
r2 = x2.shape[1]

_u = d[['state_county', 'county_uranium']].drop_duplicates()
_u['state_county'] = _u.state_county.map(lambda x: 'state_county[' + x + ']')
_u = _u.set_index('state_county')
_u = _u.loc[x2.columns]
u = patsy.dmatrix('county_uranium', data=_u, return_type='dataframe')
r3 = u.shape[1]


assert np.all(u.index == x2.columns)


with pm.Model() as model5:
    beta = pm.Normal('beta', sd=10, shape=(r1, 1))
    
    gamma_hyper_beta = pm.Normal('gamma_hyper_beta', sd=10, shape=(r3, 1))
    gamma_hyper_mu = pm.Deterministic('gamma_hyper_mu', tt.dot(u.values, gamma_hyper_beta))
    gamma_hyper_sd = pm.HalfCauchy('gamma_hyper_sd', beta=2)
    gamma_ = pm.Normal('gamma_', shape=(r2, 1))
    gamma = pm.Deterministic('gamma', gamma_hyper_mu + gamma_ * gamma_hyper_sd)
    
    mu = tt.dot(x1.values, beta) + tt.dot(x2.values, gamma)

    sigma = pm.HalfCauchy('sigma', beta=2)
    obs = pm.Normal('obs', mu=mu, sd=sigma, observed=y)

In [ ]:
with model4:
    trace4 = pm.sample(tune=1000)

In [ ]:
pm.traceplot(trace4,
             var_names=['beta', 'gamma_hyper_beta0',
                        'gamma_hyper_beta', 'gamma_hyper_sd',
                        'sigma'])

In [ ]:
pm.forestplot(trace4, var_names=['gamma'])

In [ ]:
trace4.gamma_hyper_mu.T[0]

In [ ]:
quantiles = [0.05, 0.25, 0.5, 0.75, 0.95]
gamma_dist_prd = pd.DataFrame(np.quantile(trace4.gamma_hyper_mu, quantiles, axis=0).T[0],
                              columns=quantiles, index=counties)
gamma_dist_obs = pd.DataFrame(np.quantile(trace4.gamma, quantiles, axis=0).T[0],
                              columns=quantiles, index=counties)

In [ ]:
plt.scatter(gamma_dist_prd[0.5], gamma_dist_obs[0.5])
plt.vlines(gamma_dist_prd[0.5], gamma_dist_obs[0.25], gamma_dist_obs[0.75], lw=0.5)
plt.hlines(gamma_dist_obs[0.5], gamma_dist_prd[0.25], gamma_dist_prd[0.75], lw=0.5)

## Add a random slope term by county without pooling

In [ ]:
y, x1 = patsy.dmatrices('log_radon ~ is_basement', data=d, return_type='dataframe')
x1.drop(columns=['Intercept'], inplace=True)
n, r1 = x1.shape

x2 = patsy.dmatrix('state_county - 1', data=d, return_type='dataframe')
r2 = x2.shape[1]

_u = d[['state_county', 'county_uranium']].drop_duplicates()
_u['state_county'] = _u.state_county.map(lambda x: 'state_county[' + x + ']')
_u = _u.set_index('state_county')
_u = _u.loc[x2.columns]
u = patsy.dmatrix('county_uranium', data=_u, return_type='dataframe')
r3 = u.shape[1]


assert np.all(u.index == x2.columns)


with pm.Model() as model5:
    beta = pm.Normal('beta', sd=10, shape=(r1, 1))
    
    gamma_hyper_beta = pm.Normal('gamma_hyper_beta', sd=10, shape=(r3, 1))
    gamma_hyper_mu = pm.Deterministic('gamma_hyper_mu', tt.dot(u.values, gamma_hyper_beta))
    gamma_hyper_sd = pm.HalfCauchy('gamma_hyper_sd', beta=2)
    gamma_ = pm.Normal('gamma_', shape=(r2, 1))
    gamma = pm.Deterministic('gamma', gamma_hyper_mu + gamma_ * gamma_hyper_sd)
    
    kappa = pm.Normal('kappa', sd=10, shape=(r2, 1))
    
    mu = tt.dot(x1.values, beta) + tt.dot(x2.values, gamma) + tt.dot(x1.values * x2.values, kappa)

    sigma = pm.HalfCauchy('sigma', beta=2)
    obs = pm.Normal('obs', mu=mu, sd=sigma, observed=y)

In [ ]:
with model5:
    trace5 = pm.sample(tune=2000)

In [ ]:
pm.traceplot(trace5, var_names=['beta',
                                'gamma_hyper_beta', 'gamma_hyper_sd',
                                'sigma'])

In [ ]:
pm.forestplot(trace5, var_names=['kappa'])

In [ ]:
i = -200

mu_expect5 = mu.eval({getattr(model5, k.name): trace5[i][k.name]
                      for k in model5.vars
                      if k.name not in ['sigma_log__']})
plt.scatter(mu_expect5, y.values)
plt.plot([0, 2], [0, 2])

In [ ]:
with model5:
    ppc5 = pd.DataFrame(pm.sample_posterior_predictive(trace5, samples=1)['obs'][0],
                        index=y.index, columns=y.columns)
    
plt.scatter(ppc5.log_radon, y.log_radon)
plt.plot([-1, 4], [-1, 4])

In [ ]:
plt.scatter(trace5['gamma_hyper_beta'][:,0,0],
            trace5['gamma_hyper_beta'][:,1,0])

In [ ]:
trace5.varnames

## Add a random slope term by county with partial pooling

In [ ]:
y, x1 = patsy.dmatrices('log_radon ~ is_basement', data=d, return_type='dataframe')
x1.drop(columns=['Intercept'], inplace=True)
n, r1 = x1.shape

x2 = patsy.dmatrix('state_county - 1', data=d, return_type='dataframe')
r2 = x2.shape[1]

_u = d[['state_county', 'county_uranium']].drop_duplicates()
_u['state_county'] = _u.state_county.map(lambda x: 'state_county[' + x + ']')
_u = _u.set_index('state_county')
_u = _u.loc[x2.columns]
u = patsy.dmatrix('county_uranium', data=_u, return_type='dataframe')
r3 = u.shape[1]


assert np.all(u.index == x2.columns)


with pm.Model() as model6:
    beta = pm.Normal('beta', sd=10, shape=(r1, 1))
    
    gamma_hyper_beta = pm.Normal('gamma_hyper_beta', sd=10, shape=(r3, 1))
    gamma_hyper_mu = pm.Deterministic('gamma_hyper_mu', tt.dot(u.values, gamma_hyper_beta))
    gamma_hyper_sd = pm.HalfCauchy('gamma_hyper_sd', beta=2)
    gamma_ = pm.Normal('gamma_', shape=(r2, 1))
    gamma = pm.Deterministic('gamma', gamma_hyper_mu + gamma_ * gamma_hyper_sd)
    
    kappa_hyper_beta = pm.Normal('kappa_hyper_beta', sd=10, shape=(r3, 1))
    kappa_hyper_mu = pm.Deterministic('kappa_hyper_mu', tt.dot(u.values, kappa_hyper_beta))
    kappa_hyper_sd = pm.HalfCauchy('kappa_hyper_sd', beta=2)
    kappa_ = pm.Normal('kappa_', shape=(r2, 1))
    kappa = pm.Deterministic('kappa', kappa_hyper_mu + kappa_ * kappa_hyper_sd)
    
    mu = tt.dot(x1.values, beta) + tt.dot(x2.values, gamma) + tt.dot(x1.values * x2.values, kappa)

    sigma = pm.HalfCauchy('sigma', beta=2)
    obs = pm.Normal('obs', mu=mu, sd=sigma, observed=y)

In [ ]:
with model6:
    trace6 = pm.sample(tune=1000)

In [ ]:
models_and_traces = {
                     model0: trace0,    # 0
                     model1: trace1,    # 1
                     model2: trace2,    # 2
                     model3: trace3,    # 3
                     model4: trace4,    # 4
                     model5: trace5,    # 5
                     model6: trace6,    # 6
                     }

pm.compare(models_and_traces, ic='LOO')